In [33]:
import QuantLib as ql
import openpyxl as xl

In [34]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [35]:
class DFXForward:    
    def __init__(self, evDate, maturity, position, CCYComm, NPAComm, 
        CCYTerm, NPATerm, FXSpot, h_discTSComm, h_discTSTerm):
        self._ValueDate = evDate
        self._Maturity = maturity
        if position == 'Buy':          
            self._Position = 1.0
        else:
            self._Position = -1.0
        self._CCYComm = CCYComm
        self._NPAComm = NPAComm
        self._CCYTerm = CCYTerm
        self._NPATerm = NPATerm
        self._FXSpot = FXSpot
        self._H_discTSComm = h_discTSComm
        self._H_discTSTerm = h_discTSTerm
        
    def NPV(self):
        DfComm = self._H_discTSComm.discount(self._Maturity)
        DfTerm = self._H_discTSTerm.discount(self._Maturity)
        PVComm = DfComm * self._NPAComm
        PVTerm = DfTerm * self._NPATerm
        self._Value = self._Position * (PVComm * self._FXSpot - PVTerm)
        return self._Value

In [36]:
PstBook = xl.load_workbook('Pst_Fwd.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
CommCCY = 'USD'
TermCCY = 'TWD'
Type = 'FWD'
MTMDate = MktSheet.cell(1, 3).value
FXSpot = MktSheet.cell(3, 6).value

In [37]:
settings = ql.Settings.instance()
evDate = DatetimeToDate(MTMDate)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
CommDC = ql.Actual360()
TermDC = ql.Actual365Fixed()
BDC = ql.Unadjusted
settlementDays = 0

In [38]:
CommZeroSP = MktSheet.cell(22, 6).value
CommZero3M = MktSheet.cell(23, 6).value
CommZero6M = MktSheet.cell(24, 6).value
CommZero1Y = MktSheet.cell(25, 6).value
print(CommZeroSP)

0.001459727696536763


In [39]:
DateSP = evDate
Date3M = Cal.advance(evDate, 3, ql.Months, BDC, False)
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date1Y = Cal.advance(evDate, 1, ql.Years, BDC, False)
print(DateSP)

June 30th, 2021


In [40]:
TermZeroSP = MktSheet.cell(22, 12).value
TermZero3M = MktSheet.cell(23, 12).value
TermZero6M = MktSheet.cell(24, 12).value
TermZero1Y = MktSheet.cell(25, 12).value
print(TermZeroSP)

-0.00392904297456824


In [41]:
ListDate = list([DateSP, Date3M, Date6M, Date1Y])
ListCommRate = list([CommZeroSP, CommZero3M, CommZero6M, CommZero1Y])
ListTermRate = list([TermZeroSP, TermZero3M+0.0001, TermZero6M, TermZero1Y])
ListDate
ListCommRate
ListTermRate

[-0.00392904297456824,
 -0.003829042973135284,
 -0.0006520395000015291,
 0.001329938573603508]

In [42]:
VecDate = ql.DateVector(ListDate)
VecCommRate = ql.DoubleVector(ListCommRate)
VecTermRate = ql.DoubleVector(ListTermRate)

In [43]:
TSCommZero = ql.ZeroCurve(VecDate, VecCommRate, CommDC, Cal, ql.Linear(), 
                          ql.Continuous, ql.Annual)
h_TSCommZero = ql.YieldTermStructureHandle(TSCommZero)
TSTermZero = ql.ZeroCurve(VecDate, VecTermRate, TermDC, Cal, ql.Linear(), 
                          ql.Continuous, ql.Annual)
h_TSTermZero = ql.YieldTermStructureHandle(TSTermZero)

In [44]:
FWDSheet = PstBook['FWD']

In [45]:
start_row = 2
end_row = 4
start_col = 0
end_col = 7
recordcount = end_col - start_col + 1
ListFWD = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in FWDSheet[i][start_col:end_col+1]]
    ListFWD.append(row)
    print(row)

['FWD001', datetime.datetime(2021, 5, 21, 0, 0), datetime.datetime(2021, 9, 21, 0, 0), 'Buy', 'USD', 28, 1000000, 28000000]
['FWD002', datetime.datetime(2021, 6, 15, 0, 0), datetime.datetime(2021, 12, 15, 0, 0), 'Sell', 'USD', 27.8, 1000000, 27800000]
['FWD003', datetime.datetime(2021, 6, 23, 0, 0), datetime.datetime(2022, 5, 23, 0, 0), 'Buy', 'USD', 28.1, 1000000, 28100000]


In [46]:
FWDBook = list()
for i in range(len(ListFWD)):
    row = ListFWD[i]
    record = list()
    record.append(row[0])
    record.append(DatetimeToDate(row[1]))
    record.append(DatetimeToDate(row[2]))
    record.append(row[3])
    record.append(row[4])
    record.append(row[5])
    record.append(row[6])
    record.append(row[7])
    FWDBook.append(record)

In [47]:
FWDBook

[['FWD001',
  Date(21,5,2021),
  Date(21,9,2021),
  'Buy',
  'USD',
  28,
  1000000,
  28000000],
 ['FWD002',
  Date(15,6,2021),
  Date(15,12,2021),
  'Sell',
  'USD',
  27.8,
  1000000,
  27800000],
 ['FWD003',
  Date(23,6,2021),
  Date(23,5,2022),
  'Buy',
  'USD',
  28.1,
  1000000,
  28100000]]

In [48]:
SenSheet = PstBook['FXR']
for i in range(0, len(FWDBook)):    
    record = FWDBook[i]
    tradeDate = record[1]
    maturityDate = record[2]
    position = record[3]
    CCY = record[4]
    CommAmt = record[6]
    TermAmt = record[7]            
    FWD = DFXForward(evDate, maturityDate, position, CommCCY, CommAmt, TermCCY, TermAmt, 
                     FXSpot, h_TSCommZero, h_TSTermZero)        
    Value = FWD.NPV()
    print(Value)
    
    FXSpot_1 = FXSpot * 1.01
    FWD = DFXForward(evDate, maturityDate, position, CommCCY, CommAmt, TermCCY, TermAmt, 
                     FXSpot_1, h_TSCommZero, h_TSTermZero)        
    Value_1 = FWD.NPV()
    Delta = (Value_1 - Value) * 100.0
    # print(Value_1)
    
    FXSpot_Up = FXSpot * 1.15
    FWD = DFXForward(evDate, maturityDate, position, CommCCY, CommAmt, TermCCY, TermAmt, 
                     FXSpot_Up, h_TSCommZero, h_TSTermZero)        
    Value_Up = FWD.NPV()
    # print(Value_Up)
    
    FXSpot_Dn = FXSpot * 0.85
    FWD = DFXForward(evDate, maturityDate, position, CommCCY, CommAmt, TermCCY, TermAmt, 
                     FXSpot_Dn, h_TSCommZero, h_TSTermZero)        
    Value_Dn = FWD.NPV()
    # print(Value_Dn)
    
    Curv_Up = (-((Value_Up - Value) - 0.15 * Delta))
    Curv_Dn = (-((Value_Dn - Value) + 0.15 * Delta))    
    
    SenSheet.cell(i+5, 1).value = record[0]
    SenSheet.cell(i+5, 2).value = Value
    SenSheet.cell(i+5, 3).value = CCY
    SenSheet.cell(i+5, 4).value = Delta
    SenSheet.cell(i+5, 6).value = Curv_Up
    SenSheet.cell(i+5, 8).value = Curv_Dn
    # print(Delta)
    # print(Curv_Up)
    # print(Curv_Dn)
    
PstBook.save('Pst_FWD.xlsx')
PstBook.close()

-142838.03728270903
-50667.52624470368
-249905.55978116766
